# Dashboards Deep Dive: Parameters and Filters

Build interactive dashboards with dynamic parameters and filters using your semantic model.

## What You'll Learn

✅ Create static parameters for quick dropdown filters  
✅ Use query-based parameters for dynamic values  
✅ Add interactive filters to dashboards  
✅ Build visualizations from semantic models  

---

## Why Interactive Dashboards?

Your leadership needs dashboards ready by end of week. Basic static charts won't cut it - they need:
- **Filters by factory and region** - Different stakeholders need different views
- **Date range selection** - View last 7 days, 30 days, or custom ranges
- **Dynamic updates** - As new data arrives, dashboards auto-refresh

**Parameters and filters** make this possible without creating dozens of separate dashboards.

---

## Setup: Use Your Semantic Model

In the previous notebook, you created the `sensor_metrics` metric view in your personal schema. This dashboard will query that model, ensuring consistent metrics across all visualizations.

**Reference:**
- Table: `dwx_airops_insights_platform_dev_working.{your_username}.sensor_metrics` (the metric view you created)
- Metrics: `Average Airflow Rate`, `Max Temperature`, `Total Delay`, `Record Count`
- Dimensions: `Factory ID`, `Model ID`, `Device ID`, `Date`

**Note:** Replace `{your_username}` with your actual schema name (the value shown as `WRITE_SCHEMA` in the configuration cells from previous notebooks).

---

**References:**
- [Dashboard Parameters](https://docs.databricks.com/aws/en/dashboards/tutorials/query-based-params)
- [Dashboard Filters](https://docs.databricks.com/aws/en/dashboards/filters)
- [Creating Dashboards](https://docs.databricks.com/aws/en/dashboards/)

## Overview: Building Interactive Dashboards

In this notebook, you'll transform a basic dashboard into an interactive, filterable dashboard that leadership can use to explore IoT sensor data. Here's what we'll build:

1. **Dynamic parameter queries** - Dropdown filters that populate from your data
2. **Linked visualizations** - Charts that respond to filter selections
3. **Metric view integration** - Leverage your semantic model for consistent metrics

**The goal:** Create a single dashboard that serves multiple stakeholders instead of creating separate dashboards for each device, factory, or model.

---

## 1. Edit a Dashboard

**What this does:** Opens an existing dashboard in edit mode so you can add interactive features.

**Why this matters:** You'll build on the foundation from yesterday, adding parameters and filters that make the dashboard dynamic. This is how you transform a static report into an interactive analytics tool.

1. Click **Dashboards** in the sidebar
2. Select the dashboard from yesterday
3. Click **Edit Draft**
3. Name it: `{insert_your_username} - IoT Factory Performance`

## 2: Create Dynamic Filter

**What this does:** Creates SQL queries that will populate dropdown filters with real data from your tables.

**Why this matters:** Instead of hardcoding filter values (like manually typing "Device 1, Device 2, Device 3..."), these queries automatically pull current values from your data. When new devices are added to your system, they'll automatically appear in the filter dropdown - no dashboard maintenance required!

**The pattern:** 
- **First query** (`get_device_list`) - Fetches the list of filter options (all unique device IDs)
- **Second query** (`model_performance_trend`) - Uses the selected filter value to show filtered results

Now create a query that defines the factory filter values:

In [0]:
# Query: get_device_list
# This populates the dropdown dynamically

parameter_query = """
SELECT DISTINCT 
  `Device ID`
FROM dwx_airops_insights_platform_dev_working.db_crash_course.sensor_metrics
ORDER BY `Device ID`
"""

print("Parameter query for device dropdown:")
print(parameter_query)
print("\n📋 Copy this query into a new Dataset SQL query named 'get_device_list'")

**What you just created:** A simple query that returns all unique device IDs from your metric view, sorted for easy browsing. This will become the source for your dropdown filter.

**Next step:** Create the main query that will use this filter. Notice the special `:model_id` syntax in the query below - this is a **parameter placeholder** that will be replaced by the user's dropdown selection.

## 3. Link Parameter to Query

**What this does:** Connects your filter dropdown to your visualization, creating an interactive dashboard where selecting a value instantly updates the chart.

**Why this matters:** This is the step that brings everything together. Without this linkage, your filter and visualization would exist independently. With it, you've created a dynamic, user-controlled analytics experience.

**The workflow:**
1. **Add visualization** - Create a chart from your parameterized query
2. **Add filter** - Create a dropdown populated by your parameter query
3. **Link them** - Tell the dashboard which parameter to update when the filter changes

**In the Dashboard Page Editor:**

### Add Visualization
**Dataset**: model_performance_trend  
**Scatter Chart:**
- **X-axis**: `model_id`
- **Y-axis**: `AVG(avg_delay)`
- **Chart type**: Scatter

---

### Add a filter

- **Type**: Single Value
- **Fields**: get_model_list.model_id
- **Parameters**: model_performance_trend.model_id

You can now filter the chart based on the dynamic model_id values coming from the table!

In [0]:
# Query: model_performance_trend
# This query uses the parameter from the dropdown filter

model_performance_trend = """
SELECT
  model_id,
  COUNT(*) AS total_inspections,
  AVG(defect) AS avg_defect_rate,
  SUM(count) AS total_count,
  AVG(average_temperature) AS avg_temperature,
  AVG(average_density) AS avg_density,
  AVG(average_delay) AS avg_delay,
  AVG(average_rotation_speed) AS avg_rotation_speed,
  AVG(average_air_pressure) AS avg_air_pressure
FROM dwx_airops_insights_platform_dev_working.db_crash_course.inspection_gold
WHERE model_id = :model_id OR :model_id='All'
GROUP BY model_id
ORDER BY avg_defect_rate DESC
"""

print("Query to use the model dropdown:")
print(model_performance_trend)
print("\n📋 Copy this query into a new Dataset SQL query named 'model_performance_trend'")

## 4: Use the Metric View

**What this does:** Adds your semantic model (metric view) as a data source for visualizations.

**Why this matters:** Remember the metric view you created in the previous notebook? This is where it pays off! By using the metric view:
- ✅ **Consistent metrics** - Everyone sees the same calculations for "Average Airflow Rate"
- ✅ **Simplified queries** - No need to write complex SQL with joins - the metric view handles it
- ✅ **Genie integration** - Dashboards using metric views work seamlessly with Genie's natural language queries
- ✅ **Automatic updates** - When you improve your metric definitions, all dashboards update automatically

**The difference:** Without metric views, each dashboard might calculate "average" differently. With metric views, you've created a single source of truth that Genie, dashboards, and reports all use consistently.

**In the Data Editor:**

### Add Dataset
- Click **+ Add data source**   
- Select **Metric Views**
- Select your metric view from the previous exercise


---

### Create Visualization

**In the Dashboard Page Editor:**
- Select the metric view dataset
- Ask Assistant to show `scatter plot of airflow rate over time`


## 5. Publish Your Dashboard

**What this does:** Makes your interactive dashboard live and accessible to your team and leadership.

**Why this matters:** A draft dashboard only you can see isn't useful to anyone. Publishing:
- ✅ Makes the dashboard available to stakeholders
- ✅ Enables sharing via URL
- ✅ Allows embedding in other tools
- ✅ Sets up automatic refresh schedules

**What you've accomplished:** You've created a production-ready dashboard that:
- Updates dynamically with new data
- Provides interactive filtering for different stakeholders
- Uses standardized metrics from your semantic model
- Requires zero maintenance as new devices/models are added

**Click Publish!**

---

## What You've Built

Congratulations! You now have an interactive dashboard with:

1. **Dynamic filters** - Dropdowns that automatically populate from your data
2. **Parameterized queries** - Visualizations that respond to user selections
3. **Metric view integration** - Consistent calculations used across all charts

**Real-world impact:**
- Factory managers can filter to see only their devices
- Executives can view all data or drill down to specific models
- As new data arrives, filters and charts update automatically
- One dashboard serves everyone instead of creating hundreds of static reports

**Next steps:** In the next notebook, you'll connect Genie to this semantic model so users can ask questions in natural language!

---

## Try This Out: Drill-Through Navigation

**What this does:** Enables users to click on a data point in one visualization to automatically navigate to a detailed view with filtering applied.

**Why this matters:** Imagine an executive viewing a high-level chart showing defect rates by model. With drill-through, they can click on "Model A" and instantly navigate to a detailed page showing all inspections, sensor readings, and quality metrics for just that model—without manually setting filters.

**Real-world scenario:** 
- **Overview page:** Bar chart of average defect rates by model
- **Detail page:** Comprehensive dashboard with multiple charts showing temperature trends, rotation speeds, and individual inspection records
- **The magic:** Click a bar for "Model X" → instantly see the detail page with all charts filtered to "Model X"

---

### How Drill-Through Works

**The pattern:**
1. **Source visualization** - A chart on your overview page (e.g., defect rates by model)
2. **Target page** - A detailed page with visualizations based on the same dataset
3. **Automatic filtering** - When a user clicks a data segment, the target page opens with all compatible visualizations automatically filtered

**Supported chart types for drill-through:**
- Bar charts
- Box plots
- Heatmaps
- Histograms
- Pie charts
- Scatter plots
- Point maps

---

### Setting Up Drill-Through

**Prerequisites:**
- Both the source chart and target page visualizations must use the same dataset
- Matching data types (numeric-to-numeric, categorical-to-dropdown, etc.)

**Steps:**

1. **Create your overview page** with a source chart
   - Example: Bar chart showing `AVG(avg_defect_rate)` by `model_id`

2. **Create a detail page** with multiple visualizations
   - Example: Time series of temperature, scatter plot of rotation speed, table of inspections
   - All based on the same dataset as your source chart

3. **Enable drill-through:**
   - Go back to your overview page
   - Right-click on a data segment in your source chart (e.g., a specific bar)
   - Select **"Drill to"** from the context menu
   - Choose your target detail page

4. **Test it:**
   - Click on different data points in your source chart
   - Watch as the detail page opens with automatic filtering applied!

---

### Best Practices

✅ **Keep datasets consistent** - Source and target must use the same dataset for auto-filtering to work  
✅ **Design for exploration** - Overview pages show high-level trends, detail pages show comprehensive metrics  
✅ **Add breadcrumbs** - Include a "Back to Overview" button or text box on detail pages  
✅ **Test thoroughly** - Click different data points to ensure filtering works as expected  

---

**Learn more:** [Drill-Through Documentation](https://docs.databricks.com/aws/en/dashboards/#use-drill-through-to-filter-data)